# 如何将参数从一个步骤传递到下一个步骤

:::info 前置条件

本指南假设您已熟悉以下概念：
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [链接 runnables](/docs/how_to/sequence/)
- [并行调用 runnables](/docs/how_to/parallel/)
- [自定义函数](/docs/how_to/functions/)

:::


在构建包含多个步骤的链时，有时您想要将前面步骤的数据原封不动地传递给后面的步骤使用。[`RunnablePassthrough`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html) 类可以实现这一点，它通常与 [RunnableParallel](/docs/how_to/parallel/) 一起使用，将数据传递给构建的链中的后续步骤。

请看以下示例：

In [ ]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

如上所示，`passed` 键使用了 `RunnablePassthrough()`，所以它简单地传递了 `{'num': 1}`。

我们还在映射中设置了第二个键 `modified`。它使用一个 lambda 函数将 num 加 1 来设置单个值，结果使 `modified` 键的值为 `2`。

## 检索示例

在下面的示例中，我们看到一个更实际的用例，其中我们在链中使用 `RunnablePassthrough` 和 `RunnableParallel` 来正确格式化提示的输入：

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison 在 kensho 工作"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """仅基于以下上下文回答问题：
{context}

问题：{question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("harrison 在哪里工作？")

'Harrison worked at Kensho.'

这里，提示的输入预期是一个包含 "context" 和 "question" 键的映射。用户输入仅是问题。所以我们需要使用检索器获取上下文，并将用户输入作为 "question" 键传递。`RunnablePassthrough` 允许我们将用户的问题传递给提示和模型。

## 下一步

现在您已经学会了如何在链中传递数据，以帮助格式化流经链的数据。

要了解更多信息，请参阅本节中关于 runnables 的其他操作指南。